In [1]:
import os 
import pandas as pd
import string
from typing import List
from datetime import datetime
import re
import numpy as np
import hashlib
import json
from itertools import combinations, permutations
from sklearn.preprocessing import StandardScaler
from tqdm.notebook import tqdm

In [2]:
def returnSortedCSVList(csv_directory: str) -> List:
    sortedfilelist = list()
    def sort_key(filename):
        underscoreindex = filename.index('_')
        return filename[underscoreindex+1:-4]
    for dirpath, dirnames, filenames in os.walk(csv_directory):
        dirnames.sort()
        sorted_files = sorted(filenames, key=sort_key)
        sorted_files_with_paths = [os.path.join(dirpath, f) for f in sorted_files]
        if sorted_files_with_paths:
            sortedfilelist += sorted_files_with_paths
    return sortedfilelist

In [3]:
def findFileNameTimeStamp(csv_file: str) -> str:
    short_index = csv_file.index('B')
    shortfilename = csv_file[short_index:]
    underscore_index = shortfilename.index('_')
    datestring = shortfilename[underscore_index+1:-4]
    digits = re.findall(r'\d+', datestring)
    finaldatedigits = ''.join(digits)
    return int(finaldatedigits)

In [4]:
def convertRAWcsvtoDF(csv_file: str) -> pd.DataFrame:
    rawcsvDF = pd.read_csv(csv_file)
    rawcsvDF["Closing Date"] = datetime.fromtimestamp(findFileNameTimeStamp(csv_file))
    return rawcsvDF

In [5]:
def keepColumnsforFeatureVector(csvDF: pd.DataFrame) -> pd.DataFrame:
    csvDF["Closing Date"] = pd.to_datetime(csvDF["Closing Date"]).dt.date
    slimmedDF = csvDF.loc[:,["Proposal", "Item", "Quantity", "Vendor Name", "Bidder ID", "Unit Price", "Extension", "Closing Date"]]
    return slimmedDF
   

In [6]:
def cleanDollarColumns(csvDF: pd.DataFrame) -> pd.DataFrame:
    csvDF["Unit Price"] = csvDF["Unit Price"].astype(str).str.replace(r'[^\d.]', '', regex=True).astype(float)
    csvDF["Quantity"] = csvDF["Quantity"].astype(str).str.replace(r'[^\d.]', '', regex=True).astype(float)
    csvDF["Extension"] = csvDF["Extension"].astype(str).str.replace(r'[^\d.]', '', regex=True).astype(float)
    return csvDF

In [7]:
def BidderIDtoNumber(id_string: str)-> int:
    hash_object = hashlib.sha256(id_string.encode())
    hash_hex = hash_object.hexdigest()
    return int(hash_hex[:16], 16)
def convertBidderIDColumntoNumberID(csvDF: pd.DataFrame) -> pd.DataFrame:
    csvDF["Bidder ID"] = csvDF["Bidder ID"].astype(str).apply(BidderIDtoNumber)
    return csvDF

In [8]:
def proposalVendorNameGrouping(csvDF: pd.DataFrame):
    return csvDF.groupby('Vendor Name')

In [9]:
def VendorNameBidderIDdict(csvDF: pd.DataFrame, vendorNameBidderIDmapping: dict) -> dict:
    groupedVendors = proposalVendorNameGrouping(csvDF)
    for name, group in groupedVendors:
        vendorNameBidderIDmapping[name] = group["Bidder ID"].iloc[0]
    return vendorNameBidderIDmapping

In [10]:
def getProposalCode(csvDF: pd.DataFrame) -> str:
    return csvDF['Proposal'].iloc[0]

def getClosingDate(csvDF: pd.DataFrame) -> int:
    return round(csvDF["Closing Date"].iloc[0].toordinal())

def getBidderID(csvDF: pd.DataFrame) -> int:
    return csvDF["Bidder ID"].iloc[0]

def getContractorTotalBidAmount(csvDF: pd.DataFrame) -> float:
    return csvDF["Extension"].sum()

def getContractorName(csvDF: pd.DataFrame) -> str:
    return csvDF["Vendor Name"].iloc[0]

def getNumberofContractorsBidding(groupofBidders) -> int:
    return len(groupofBidders)

In [11]:
def combineDuplicatedProposalItems(csvDF: pd.DataFrame) -> pd.DataFrame:

    def process_group(group):
        duplicatedItems = group[group["Item"].duplicated(keep=False)]
        
        if duplicatedItems.empty:
            return group
    
        grouped = duplicatedItems.groupby("Item").agg({
            "Quantity": "sum",
            "Unit Price": lambda x: np.average(x, weights=duplicatedItems.loc[x.index, "Quantity"]), # weighted average
            "Extension": "sum"
        }).reset_index()
        
        mergedRows = duplicatedItems.drop_duplicates(subset="Item").merge(
            grouped, on="Item", suffixes=('', '_new')
        )
        
        mergedRows["Quantity"] = mergedRows["Quantity_new"]
        mergedRows["Unit Price"] = mergedRows["Unit Price_new"]
        mergedRows["Extension"] = mergedRows["Extension_new"]
        mergedRows = mergedRows.drop(columns=["Quantity_new", "Unit Price_new", "Extension_new"])
        
        withoutDuplicatedDF = group[~group["Item"].duplicated(keep='first')].dropna(subset=["Item"])
        
        finalDF = pd.concat([withoutDuplicatedDF, mergedRows]).drop_duplicates(subset="Item", keep="last")
        
        return finalDF
         
    vendorNamegrouping = csvDF.groupby("Vendor Name")
    processed_groups = []
    for name, group in vendorNamegrouping:
        processed_group = process_group(group)
        processed_groups.append(processed_group)


    finalDF = pd.concat(processed_groups).reset_index(drop=True)

    
    return finalDF

In [12]:
def contractorDesiredCompetitorPerms(proposalcontractors: dict):
    allVendors = list(proposalcontractors.keys())
    vendorPerms = list(permutations(allVendors, 2))
    return vendorPerms
#proposalcontractors = {name: group.to_dict('records') for name, group in duplicatesRemovedBidTab.groupby("Vendor Name")}

#result = contractorDesiredCompetitorPerms(proposalcontractors)
#print(result)
#print(len(proposalcontractors))

In [13]:
def combinewithGADOTLineItems(csvDF: pd.DataFrame) -> dict:
    contractorsINproposal = dict()
    vendorNamegrouping = proposalVendorNameGrouping(csvDF)
    GADOT_ITEMS = pd.read_csv("../PayItemIndex_2021.csv").drop(columns=["UNITS", "LS UNITS", "DESCRIPTION"])
    proposalcode = getProposalCode(csvDF)
    closingdate = getClosingDate(csvDF)
    for name, group in vendorNamegrouping:
        bidderID = getBidderID(group)
        contractorname = getContractorName(group)
        numberofContractorsBidding = getNumberofContractorsBidding(vendorNamegrouping)
        if numberofContractorsBidding == 1: # don't want contracts with only one bidder
            continue
        mergedAllItems = pd.merge(GADOT_ITEMS[GADOT_ITEMS["ITEM NO."].isin(eligiblelineitemsEverBidon)], group, 
                          left_on="ITEM NO.", right_on="Item", how="left")

        # Sort the DataFrame based on "ITEM NO."
        mergedAllItems = mergedAllItems.sort_values("ITEM NO.")
    
        # Now drop the unnecessary columns and reset the index
        mergedAllItems = mergedAllItems.drop(columns=["Proposal", "Vendor Name", "Bidder ID", "Closing Date", "Item", "ITEM NO."]).reset_index(drop=True)
        mergedAllItems = mergedAllItems.astype(float).fillna(0)
       
        #print(f"Len Merged All Items: {len(mergedAllItems)}")
        totalBidAmount = getContractorTotalBidAmount(mergedAllItems)
        mergedAllItems = mergedAllItems.drop(columns=["Extension"])
        #mergedAllItems = extractOnlyItemsEverBidOn(mergedAllItems, eligiblelineitemsEverBidon)
        #print(f"Potential Line items: {len(mergedAllItems)}")
        contractorsINproposal[contractorname] = {"Line Items":mergedAllItems}
    
    group_one_hot = get_one_hot_encodingGroupofContractors(vendorNamegrouping.groups.keys())
    
    for contractor, lineitems in contractorsINproposal.items():
        singlecontractor_one_hot = get_one_hot_encodingSingleContractor(contractor)
        contractorsINproposal[contractor]["Contractor One Hot"] = singlecontractor_one_hot
        contractorsINproposal[contractor]["Competitors One Hot"] = group_one_hot
        contractorsINproposal[contractor]["Number of Competitors"] = numberofContractorsBidding
        contractorsINproposal[contractor]["Quantity"] = contractorsINproposal[contractor]["Line Items"]["Quantity"]
        contractorsINproposal[contractor]["Unit Price"] = contractorsINproposal[contractor]["Line Items"]["Unit Price"]
        
        #print(f"Length of solo one hot {len(singlecontractor_one_hot)}")
        #print(f"{contractor}: {contractorsINproposal[contractor]['Line Items'].shape}")
        #print(contractorsINproposal[contractor]["Line Items"][contractorsINproposal[contractor]["Line Items"]["Quantity"] > 0])
        
   
    #print(f"Length of group one hot: {len(group_one_hot)}")
    #print(f"Contractors Encountered: {len(vendorNameIDmapping)}")
    
    return contractorsINproposal
        

In [14]:
def createXexample(contractor: str, competitor: str, proposalcontractors: dict, proposalid: str, timestamp: int, countyonehot: pd.DataFrame) -> pd.DataFrame:
    contractor_arr = np.array(proposalcontractors[contractor]["Contractor One Hot"], dtype=np.uint8)
    desired_competitor_arr = np.array(get_one_hot_encodingSingleContractor(competitor), dtype=np.uint8)
    numcompetitors_arr = np.array([proposalcontractors[contractor]["Number of Competitors"]], dtype=np.uint8)
    competitors_arr = np.array(proposalcontractors[contractor]["Competitors One Hot"], dtype=np.uint8)
    timestamp_arr = np.array([timestamp], dtype=np.int64)
    counties_arr = np.array(countyonehot[proposalid], dtype=np.uint8)
    quantities = np.array(proposalcontractors[contractor]["Quantity"], dtype=np.float64)
    #print(f"One Hot Length: {len(contractor_arr)+len(desired_competitor_arr)+len(competitors_arr)+len(counties_arr)}") # 748 on is numerical

    # Concatenate all arrays
    feature_list = [
        proposalid,
        contractor,
        competitor,
        *contractor_arr,
        *desired_competitor_arr,
        *competitors_arr,
        *counties_arr,
        *timestamp_arr,
        *numcompetitors_arr,
        *quantities
    ]
 
    # Create column names
    column_names = [
        'ProposalID',
        'Contractor',
        'Competitor',
        *[f'ContractorOneHot_{i}' for i in range(len(contractor_arr))],
        *[f'DesiredCompetitorOneHot_{i}' for i in range(len(desired_competitor_arr))],
        *[f'CompetitorsOneHot_{i}' for i in range(len(competitors_arr))],
        *[f'County_{i}' for i in range(len(counties_arr))],
         'Timestamp',
        'NumberOfCompetitors',
        *[f'Quantity_{i}' for i in range(len(quantities))]
    ]

    # Create DataFrame
    df = pd.DataFrame([feature_list], columns=column_names)

    return df
    

In [15]:
def XandYexamplesfromProposal(proposalcontractors: pd.DataFrame, proposal_id: str, timestamp: int,
                          countyonehot: pd.DataFrame) -> dict:
    Xexamplesfromproposal = dict()
    Yexamplesfromproposal = dict()
    for contractor, competitor in contractorDesiredCompetitorPerms(proposalcontractors):
        single_example = createXexample(contractor, competitor, proposalcontractors, proposal_id, timestamp, countyonehot)
        single_example = single_example.iloc[0]

        single_y_label = createYlabel(contractor, competitor, proposalcontractors, proposal_id)
        single_y_label = single_y_label.iloc[0]
         # Need to create unique key for dictionary
        Xhash_object = hashlib.sha256((proposal_id + contractor + competitor).encode())
        Xhash_hex = Xhash_object.hexdigest()
        Xexamplesfromproposal[Xhash_hex] = single_example

        Yhash_object = hashlib.sha256((proposal_id + contractor + competitor).encode())
        Yhash_hex = Yhash_object.hexdigest()
        Yexamplesfromproposal[Yhash_hex] = single_y_label
    return Xexamplesfromproposal, Yexamplesfromproposal

In [16]:
def createYlabel(contractor: str, competitor: str, proposalcontractors: dict, proposal_id:str) -> dict:
    unitprices = np.array(proposalcontractors[competitor]["Unit Price"], dtype=np.float64)

    feature_list = [
        proposal_id,
        unitprices
    ]
    column_names = [
        'Proposal_id',
        'Unit Price'
    ]

    df = pd.DataFrame([feature_list], columns=column_names)
    return df


    

In [17]:
def get_one_indices(multi_hot_vector):
    return np.where(np.array(multi_hot_vector) == 1)[0]

In [18]:
def OneHotCounties() -> pd.DataFrame:
    """Provide Proposal Code and it returns the one hot encoding of it's counties"""
    with open('../proposalsCounties.json', 'r') as file:
        proposals_counties = json.load(file)
    with open('../AllGAcounties.txt', 'r') as file:
        all_counties = [line.strip() for line in file]

    one_hot_encoded = {}
    for proposal, counties in proposals_counties.items():
        counties = counties.split()
        encoded = {county.lower(): 0 for county in all_counties}
        for county in counties:
            if county.lower() in encoded:
                encoded[county.lower()] = 1
        one_hot_encoded[proposal] = encoded
    return pd.DataFrame.from_dict(one_hot_encoded, orient='index').T

In [19]:
def get_one_hot_encodingSingleContractor(contractorname):
    
    # Create a dictionary with all contractors set to 0
    one_hot_dict = {f'{cont}': 0 for cont in vendorNameIDmapping.keys()}
    
    # Set the correct contractor to 1
    one_hot_dict[f'{contractorname}'] = 1
    
    # Create the DataFrame all at once
    one_hot = pd.Series(one_hot_dict)
    
    return one_hot

In [20]:
def get_one_hot_encodingGroupofContractors(listofCompetitors):
    one_hot_dict = {f'{cont}': 0 for cont in vendorNameIDmapping.keys()}
    for contractor in listofCompetitors:
        one_hot_dict[f'{contractor}'] = 1
    one_hot = pd.Series(one_hot_dict)

    return one_hot

In [21]:
def findItemsBidOn(csvDF: pd.DataFrame) -> set:
    cleanedBidTab = cleanDollarColumns(csvDF)
    itemsbidon = set(cleanedBidTab[cleanedBidTab["Quantity"] > 0]["Item"])
    #print(f"Number of Items bid on: {len(itemsbidon)}")
    return itemsbidon
def getallGADOTItems() -> set:
    GADOT_ITEMS = pd.read_csv("../PayItemIndex_2021.csv").drop(columns=["UNITS", "LS UNITS", "DESCRIPTION"])
    items = set(GADOT_ITEMS["ITEM NO."])
    #print(f"Number of GADOT Items: {len(items)}")
    return items

def extractOnlyItemsEverBidOn(proposallineitems: pd.DataFrame, eligiblelineitems: set) -> pd.DataFrame:
    return proposallineitems[proposallineitems["Item"].isin(eligiblelineitems)]

In [22]:
## MAIN ##

sortedfiles = returnSortedCSVList('../ga_csv/')
allproposalsFeatureVectors = []
lineitemsEverBidOn = set()
vendorNameIDmapping = dict()
proposal_county_one_hot = OneHotCounties()
for file in sortedfiles:
    rawBidTab = convertRAWcsvtoDF(file)
    lineitemsEverBidOn |= findItemsBidOn(rawBidTab)
    vendorNameIDmapping = VendorNameBidderIDdict(rawBidTab, vendorNameIDmapping)
eligiblelineitemsEverBidon = getallGADOTItems() & lineitemsEverBidOn
itemsneverBidOn = getallGADOTItems() - eligiblelineitemsEverBidon
eligiblelineitemsEverBidon = sorted(eligiblelineitemsEverBidon)
eligiblelineitemsEverBidonDF = pd.Series(sorted(eligiblelineitemsEverBidon))
eligiblelineitemsEverBidonDF.to_csv("../eligibleBidItems.csv", index=False)

#print(f"All Items Ever bid on: {len(eligiblelineitemsEverBidon)}")
#print(f"Never Bid On: {len(itemsneverBidOn)}")

In [23]:
X_examples = dict()
y_labels = dict()
i = 0
for file in tqdm(sortedfiles):
    rawBidTab = convertRAWcsvtoDF(file)
    neededcolBidTab = keepColumnsforFeatureVector(rawBidTab)
    cleanedBidTab = cleanDollarColumns(neededcolBidTab)
    bidderIDConvertDF = convertBidderIDColumntoNumberID(cleanedBidTab)
    
    proposalcode = getProposalCode(bidderIDConvertDF)
    closingdate = getClosingDate(bidderIDConvertDF)
    
    duplicatesRemovedBidTab = combineDuplicatedProposalItems(bidderIDConvertDF)
    #print(duplicatesRemovedBidTab.shape)
    proposalcontractors = combinewithGADOTLineItems(duplicatesRemovedBidTab)
    

    proposalexamples, proposallabels = XandYexamplesfromProposal(proposalcontractors, proposalcode, closingdate, proposal_county_one_hot)

    #proposalexamplesDF = pd.DataFrame.from_dict(proposalexamples, orient='index')
   # proposallabelsDF = pd.DataFrame.from_dict(proposallabels, orient='index')
   # print(proposalexamplesDF.head(1))
   # print(proposallabelsDF.head(1))
    X_examples[proposalcode] = proposalexamples
    y_labels[proposalcode] = proposallabels
  

  0%|          | 0/1340 [00:00<?, ?it/s]

In [24]:
def splitIndexLocation(X_data_dict: dict, trainingPercent: float):
    listofbids = []
    for proposalid, example in X_data_dict.items():
        for rowhash, row in example.items():
            listofbids.append(row["ProposalID"])
    indexlocation = int(len(listofbids) * trainingPercent)
    while listofbids[indexlocation+1] == listofbids[indexlocation]:
        indexlocation -=1
    indexlocation+=1
    for i in range(indexlocation-5, indexlocation+5):
        if i == indexlocation:
            print(f"Boundary {listofbids[i]}")
        else:
            print(listofbids[i])
    return indexlocation


In [25]:
X_data = []
y_data = []

for proposalid, example in X_examples.items():
    for rowhash, row in example.items():
        X_data.append(example[rowhash])
        y_data.append(y_labels[proposalid][rowhash]) # same hashes

In [26]:
len(X_data) == len(y_data)

True

In [27]:
print(y_data[:10])

[Proposal_id                                       B1CBA1800717-0
Unit Price     [0.0, 0.0, 0.0, 0.0, 0.0, 1395084.78, 0.0, 0.0...
Name: 0, dtype: object, Proposal_id                                       B1CBA1800717-0
Unit Price     [0.0, 0.0, 0.0, 0.0, 0.0, 758400.0, 0.0, 0.0, ...
Name: 0, dtype: object, Proposal_id                                       B1CBA1800717-0
Unit Price     [0.0, 0.0, 0.0, 0.0, 0.0, 726545.0, 0.0, 0.0, ...
Name: 0, dtype: object, Proposal_id                                       B1CBA1800717-0
Unit Price     [0.0, 0.0, 0.0, 0.0, 0.0, 750000.0, 0.0, 0.0, ...
Name: 0, dtype: object, Proposal_id                                       B1CBA1800717-0
Unit Price     [0.0, 0.0, 0.0, 0.0, 0.0, 758400.0, 0.0, 0.0, ...
Name: 0, dtype: object, Proposal_id                                       B1CBA1800717-0
Unit Price     [0.0, 0.0, 0.0, 0.0, 0.0, 726545.0, 0.0, 0.0, ...
Name: 0, dtype: object, Proposal_id                                       B1CBA1800717-0
Unit Price

In [28]:
len(X_data)

15782

In [29]:
splitlocation = splitIndexLocation(X_examples, .9)

X_train = X_data[:splitlocation]
y_train = y_data[:splitlocation]

X_test = X_data[splitlocation:]
y_test = y_data[splitlocation:]

len(X_train), len(y_train), len(X_test), len(y_test)

B1CBA2302153-0
B1CBA2302153-0
B1CBA2302153-0
B1CBA2302153-0
B1CBA2302153-0
Boundary B1CBA2302248-0
B1CBA2302248-0
B1CBA2302248-0
B1CBA2302248-0
B1CBA2302248-0


(14196, 14196, 1586, 1586)

In [30]:
# Turn into final vectors labels and examples
y_test = pd.DataFrame(y_test)["Unit Price"].tolist()
y_train = pd.DataFrame(y_train)["Unit Price"].tolist()
y_test = np.array(y_test)
y_train = np.array(y_train)
y_test.shape, y_train.shape

((1586, 1890), (14196, 1890))

In [31]:
X_test = pd.DataFrame(X_test).iloc[:, 3:].to_numpy()
X_train = pd.DataFrame(X_train).iloc[:, 3:].to_numpy()

In [32]:
X_test.shape, X_train.shape

((1586, 2640), (14196, 2640))

In [33]:
np.save("../VAE_Data/X_train.npy", X_train, allow_pickle=True)
np.save("../VAE_Data/X_test.npy", X_test, allow_pickle=True)
np.save("../VAE_Data/y_train.npy", y_train, allow_pickle=True)
np.save("../VAE_Data/y_test.npy", y_test, allow_pickle=True)